In [ ]:
import nltk
import os
from warnings import filterwarnings
filterwarnings("ignore")
os.environ['JAVAHOME'] = 'C://Users//felix//jdk-12.0.2'

In [ ]:
# load the data
# with open("review_0408.txt","r") as f:
#     lines = [l.strip()[0:-2] for l in f.readlines()]
#     lines = [l.lower() for l in lines if l!=""]
# print(len(lines))

with open("movies_com.txt","r") as f:
    lines = [l.strip() for l in f.readlines()]
    lines = [l.lower() for l in lines if l!=""]
print(len(lines))

In [ ]:
#define grammar rules of NPs
from nltk.chunk.regexp import *
chunk_rules = ChunkRule("<.*>+", "chunk everything")
reg_parser = RegexpParser('''
        NP: {<JJ>* <NN>* <NNS>*}                           
''')

for line in lines[0:10]:
    res = reg_parser.parse(nltk.pos_tag(nltk.word_tokenize(line)))
    for row in res:
        print(row)
    print()
    res.draw()

In [ ]:
# self defined grammer, dependency relation
# select all the NPs as candidate
nps=[]
for line in lines:
    res = reg_parser.parse(nltk.pos_tag(nltk.word_tokenize(line)))
    for row in res:
        if str(type(row))!="<class 'tuple'>":
            phrase = [w[0] for w in row]
#             print(phrase)
            nps.append(" ".join(phrase))
    #print()

nps = set(nps)
print(len(nps))

In [ ]:
# three steps filtrion to find out movie names as a chunk
# the NPs frequency as first filtrition
phrase_count = {}
for phrase in nps:
    if len(phrase.split())==1:
        count = sum([1 if phrase in line.split() else 0 for line in lines])
    else:
        count = sum([1 if phrase in line else 0 for line in lines])
    phrase_count[phrase]=count

# treshold equal to 10
thres = 10
phrases = []
for key,val in phrase_count.items():
    if val>=thres:
        phrases.append(key)

print(len(phrases))
print(phrases)


# second filtrison by removing repeated NPs 
movies = []
for i in phrases:
    flag=True
    for j in phrases:
        if i==j:
            continue
        if i in j.split():
            flag=False
            break
    if flag:
        movies.append(i)
print()
print(len(movies))
print(movies)

# third filtrison, removie common modifiers 
compare_movies=[]
while len(movies)!=len(compare_movies):
    max_count = 0
    mcount = {}
    for m in movies:
        count=0
        for n in movies:
            if m==n:
                continue
            for line in lines:
                if m in line and n in line:
                    count+=1
                    break
        mcount[m]=count
    max_count = max(list(mcount.values()))
    for m,count in mcount.items():
        if count>len(movies)*0.5 and count==max_count:
            continue
        else:
            compare_movies.append(m)
    if len(movies)!=len(compare_movies):
        movies = compare_movies.copy()
        compare_movies=[]
print()
print(len(compare_movies))
print(compare_movies)

In [ ]:
# defined rules to extract comparative relations
def countvs(a,b):
    total = 0.0
    better,worse=0.0,0.0
    for line in lines:
        pa=line.find(a)
        pb=line.find(b)
        if pa==-1 or pb==-1:
            continue
        nott=line.find("not ")
        for x in ["better","best","more","prefer","won","win","beat","outstrip","defeat","outperform","top","first rate","more enjoyable","more fascinating","more pleasant","absorbing"]:
            r1=line.find(x)
            if r1!=-1:
                if nott==-1:
                    if pa<r1 and pb>r1:
                        better+=1
                    elif pb<r1 and pa>r1:
                        worse+=1
                elif nott!=-1 and nott<r1:
                    if pa<r1 and pb>r1:
                        worse+=1
                    elif pb<r1 and pa>r1:
                        better+=1
        for x in ["lack","lacking","inferior","worse","less","least","second-rate", "third-rate", "outdated",'silly','boring','disappointing',"weak"]:
            r2=line.find(x)
            if r2!=-1:
                if nott==-1:
                    if pa<r1 and pb>r1:
                        worse+=1
                    elif pb<r1 and pa>r1:
                        better+=1
        total+=1
    #print(better,worse,total)
    return (better,worse,total)
                
compare = {}
for a in compare_movies:
    val = 0 #tie
    for b in compare_movies:
        if a==b or a+'_'+b in compare or b+'_'+a in compare:
            continue
        result = countvs(a,b)
        if result[2]>0:
            print(a+" vs. "+b)
            print('better','worse','total :',result)
            compare[a+'_'+b]=(result[0]/result[2],result[1]/result[2])
            print("there are {:.2%}/{:.2%} people think {} is better/worse than {}.".format(compare[a+'_'+b][0],compare[a+'_'+b][1],a,b))
    